In [5]:
import numpy as np

In [6]:
def ground_state(vars):
    x, y = vars
    s = 3.5
    return [x / np.tan(x) + y, x**2 + y**2 - s**2]

def excited_state(vars):
    x, y = vars
    s = 3.5
    return [1 / (x * np.tan(x)) - 1 / x**2 - 1 / y - 1 / y**2, x**2 + y**2 - s**2]

In [7]:
def broyden_method(f, x_0, B_0, tol_f=1e-7, tol_s=1e-7, max_iter=300):
    x_k = np.array(x_0, dtype=float)
    B_k = B_0
    F_k = f(x_0)
    y_k = 0

    for _ in range(max_iter):

        s_k = -np.linalg.solve(B_k, F_k)
        x_k += s_k
        F_kp1 = f(x_k)
        if (np.linalg.norm(F_kp1) < tol_f) and (np.linalg.norm(s_k) < tol_s):
            break

        y_k = np.subtract(F_kp1, F_k)
        F_k = F_kp1

        B_k += np.outer((y_k - B_k @ s_k), s_k) / np.dot(s_k, s_k)

    return x_k

In [ ]:
def newton_method(f, x_0, tol_f=1e-7, tol_s=1e-7, max_iter=300):
    x_k = np.array(x_0, dtype=float)

    for _ in range(max_iter):
        J = np.zeros((len(x_k), len(x_k)))
        h = 1e-6  
        F_k = f(x_k)
        for i in range(len(x_0)):
            x_temp = x_k.copy()
            x_temp[i] += h
            J[:, i] = (np.subtract(f(x_temp), F_k)) / h  # Finite difference approximation of derivatives 
        
        try:
            s_k = -np.linalg.solve(J, F_k)
        except np.linalg.LinAlgError:
            print("Jacobian is singular.")
            break
        x_k += s_k
        if (np.linalg.norm(F_k) < tol_f) and (np.linalg.norm(s_k) < tol_s):
            break
    return x_k

In [ ]:
initial = [2.0, 2.0]
better_B0 = np.array([[-3, 1],
                      [4, 4]], dtype=float)

ground_broyden_better = broyden_method(ground_state, initial, B_0=better_B0)
ground_broyden_id = broyden_method(ground_state, initial, B_0=np.eye(2))
ground_state_newton = newton_method(ground_state, initial)


print("Ground state solution (Broyden with identity initial):", ground_broyden_id)
print("Ground state solution (Broyden with better initial):", ground_broyden_better)
print("Ground state solution (Newton's method):", ground_state_newton)


Ground state solution (Broyden with identity initial): [31.31962016  6.22789799]
Ground state solution (Broyden with better initial): [ 1.7930768  -2.48813556]
Ground state solution (Newton's method): [-323.52277457 -160.05503149]


In [37]:
initial = [3.0, 1.0]
better_B0 = np.array([[-3, 1],
                      [4, 4]], dtype=float)

excited_broyden_better = broyden_method(excited_state, initial, B_0=better_B0)
excited_broyden_id = broyden_method(excited_state, initial, B_0=np.eye(2))
excited_state_newton = newton_method(excited_state, initial)

print("Excited state solution (Broyden with identity initial):", ground_broyden_id)
print("Excited state solution (Broyden with better initial):", excited_broyden_better)
print("Excited state solution (Newton's method):", excited_state_newton)

Excited state solution (Broyden with identity initial): [19.45657686 24.89583553]
Excited state solution (Broyden with better initial): [71.0284306  -3.40403943]
Excited state solution (Newton's method): [3.31194065 1.1318344 ]


## Summary 
Here I use three methods:
1. Broyden method with bad(relatively) initial B_0 identity
2. Broyden method with good(relatively) initial B_0 achieved by manually compute the Jacobi
3. Newton's method with finite difference approximation


When the initial value is close to the excat solution, good Broyden and Newton's method has good accuracy for ground state, while bad Broyden is unstable. When initial value is not close to the exact solution, all methods cannot guarante stability.


The first excitation state is more sensitive to initial values since it contains more nonlinearity, and when the initial value is close to the exact solution, only Newton's method shows stability.